In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd

In [0]:
path = "/content/drive/My Drive/Churn_Modelling.csv"
df = pd.read_csv(path)

In [7]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [0]:
import numpy as np
import matplotlib.pyplot as plt

In [9]:
df.shape

(10000, 14)

In [0]:
X = df.iloc[:, 3:13].values
y = df.iloc[:, 13].values

In [0]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [0]:
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

In [13]:
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:,1:]

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [14]:
print(y[:10])

[1 0 1 0 0 1 0 1 0 0]


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [0]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [17]:
import keras 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense 
from tensorflow.keras.layers import Dropout 

Using TensorFlow backend.


In [23]:
classifier = Sequential()
classifier.add(Dense(units = 6, kernel_initializer = "uniform", activation = "relu", input_dim = 11 ))
classifier.add(Dense(units = 6, kernel_initializer = "uniform", activation = "relu"))
classifier.add(Dense(units = 1, kernel_initializer = "uniform", activation = "sigmoid"))
classifier.compile(optimizer = "adam",  loss = 'binary_crossentropy' , metrics=['accuracy'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [24]:
classifier.fit(X_train, y_train, batch_size = 10, nb_epoch = 100)

Epoch 1/100
8000/8000 [==============================] - 3s 346us/sample - loss: 0.4848 - acc: 0.7952
Epoch 2/100
8000/8000 [==============================] - 2s 252us/sample - loss: 0.4251 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 2s 240us/sample - loss: 0.4172 - acc: 0.8221
Epoch 4/100
8000/8000 [==============================] - 2s 236us/sample - loss: 0.4124 - acc: 0.8317
Epoch 5/100
8000/8000 [==============================] - 2s 237us/sample - loss: 0.4086 - acc: 0.8342
Epoch 6/100
8000/8000 [==============================] - 2s 238us/sample - loss: 0.4060 - acc: 0.8351
Epoch 7/100
8000/8000 [==============================] - 2s 233us/sample - loss: 0.4037 - acc: 0.8338
Epoch 8/100
8000/8000 [==============================] - 2s 242us/sample - loss: 0.4025 - acc: 0.8360
Epoch 9/100
8000/8000 [==============================] - 2s 246us/sample - loss: 0.4014 - acc: 0.8359
Epoch 10/100
8000/8000 [==============================] - 2s 241us/sample - loss: 

In [27]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)
print(y)

[1 0 1 ... 1 1 0]


In [0]:
new_prediction = classifier.predict(sc.transform(np.array([[0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction = (new_prediction > 0.5)

In [0]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(6, input_dim = 11, kernel_initializer = 'glorot_uniform', activation = 'relu' ))
    classifier.add(Dense(6, kernel_initializer = 'glorot_uniform', activation = 'relu' ))
    classifier.add(Dense(1, kernel_initializer = 'glorot_uniform', activation = 'sigmoid' ))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, nb_epoch = 100)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variable = accuracies.std()

In [31]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
def build_classifier(optimzer):
    classifier = Sequential()
    classifier.add(Dense(6, input_dim = 11, kernel_initializer = 'glorot_uniform', activation = 'relu' ))
    classifier.add(Dense(6, kernel_initializer = 'glorot_uniform', activation = 'relu' ))
    classifier.add(Dense(1, kernel_initializer = 'glorot_uniform', activation = 'sigmoid' ))
    classifier.compile(optimizer = optimzer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [25, 32],
              'nb_epoch': [100, 500],
              'optimzer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator = classifier, param_grid = parameters, scoring = 'accuracy', cv = 10)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_







Epoch 1/1
7200/7200 [==============================] - 1s 166us/step - loss: 0.5417 - acc: 0.7629
Epoch 1/1
7200/7200 [==============================] - 1s 176us/step - loss: 0.6047 - acc: 0.7053
Epoch 1/1
7200/7200 [==============================] - 1s 174us/step - loss: 0.5404 - acc: 0.7769
Epoch 1/1
7200/7200 [==============================] - 1s 178us/step - loss: 0.7025 - acc: 0.5569
Epoch 1/1
7200/7200 [==============================] - 1s 186us/step - loss: 0.6425 - acc: 0.6949
Epoch 1/1
7200/7200 [==============================] - 1s 190us/step - loss: 0.5542 - acc: 0.7768
Epoch 1/1
7200/7200 [==============================] - 1s 197us/step - loss: 0.5104 - acc: 0.7969
Epoch 1/1
7200/7200 [==============================] - 2s 210us/step - loss: 0.6455 - acc: 0.6336
Epoch 1/1
7200/7200 [==============================] - 2s 216us/step - loss: 0.6878 - acc: 0.6075
Epoch 1/1
7200/7200 [==============================] - 2s 219us/step - loss: 0.5452 - acc: 0.7706
Epoch 1/1
7200